# 3.2


In [1]:
def skewness(samples):

    # média e desvio
    std = np.std(samples)
    mean = np.mean(samples)

    # desvios da média
    deviation = np.array([x - mean for x in samples])

    return np.mean(deviation**3)/std**3


In [2]:
def kurtosis(samples):

    # média e desvio
    std = np.std(samples)
    mean = np.mean(samples)

    # desvios da média
    deviation = np.array([x - mean for x in samples])

    return np.mean(deviation**4)/std**4


Model 2

In [3]:
import sympy as sp
import numpy as np

x = sp.symbols('x', real=True)
F_custom = sp.Piecewise(
    (0, x < 0),
    (1 - (2000 / (x + 2000))**3, True),
)
pdf_custom = sp.simplify(sp.diff(F_custom, x))

F_custom_fn = sp.lambdify(x, F_custom, 'numpy')
pdf_custom_fn = sp.lambdify(x, pdf_custom, 'numpy')

print("CDF (simbolica):", F_custom)
print("PDF (simbolica):", pdf_custom)
xs = np.array([-1000, -1, 0, 1, 10, 100, 1000])
print("Valores CDF:", F_custom_fn(xs))
print("Valores PDF:", pdf_custom_fn(xs))



CDF (simbolica): Piecewise((0, x < 0), (1 - 8000000000/(x + 2000)**3, True))
PDF (simbolica): Piecewise((0, x < 0), (24000000000/(x + 2000)**4, True))
Valores CDF: [0.         0.         0.         0.0014985  0.01485124 0.1361624
 0.7037037 ]
Valores PDF: [0.         0.         0.0015     0.001497   0.00147037 0.00123405
 0.0002963 ]


In [4]:
import numpy as np

def sample_custom(size=1, eps=1e-12, rng=None):
    rng = np.random.default_rng(rng)
    u = rng.random(size)
    u = np.clip(u, 0.0, 1.0 - eps)       # evita u=1 -> inf
    return 2000 / np.power(1.0 - u, 1/3) - 2000

# amostragem da distribuicao
samples_model_2 = sample_custom(10000, rng=42)



In [5]:
print(np.std(samples_model_2))

1770.317522873039


In [6]:
print(skewness(samples_model_2))

10.665960352573526


In [7]:
print(kurtosis(samples_model_2))

267.1173595521514


In [8]:
import sympy as sp
import numpy as np

x = sp.symbols('x', real=True)
F_custom = sp.Piecewise(
    (0, x < 0),
    (1 - 0.3 * sp.exp(-0.00001 * x), True),
)
pdf_custom = sp.simplify(sp.diff(F_custom, x))

F_custom_fn = sp.lambdify(x, F_custom, 'numpy')
pdf_custom_fn = sp.lambdify(x, pdf_custom, 'numpy')

xs = np.array([-1, 0, 1, 1e3, 1e5])
print("CDF (simbolica):", F_custom)
print("PDF (simbolica):", pdf_custom)
print("Valores CDF:", F_custom_fn(xs))
print("Valores PDF:", pdf_custom_fn(xs))


CDF (simbolica): Piecewise((0, x < 0), (1 - 0.3*exp(-1.0e-5*x), True))
PDF (simbolica): Piecewise((0, x < 0), (3.0e-6*exp(-1.0e-5*x), True))
Valores CDF: [0.         0.7        0.700003   0.70298505 0.88963617]
Valores PDF: [0.00000000e+00 3.00000000e-06 2.99997000e-06 2.97014950e-06
 1.10363832e-06]


In [9]:
import numpy as np

def sample_mixture(size=1, eps=1e-12, rng=None):
    rng = np.random.default_rng(rng)
    u = rng.random(size)
    # parte discreta (massa em 0)
    x = np.zeros(size)
    mask = u >= 0.7  # 30% vão para a cauda contínua
    u_tail = u[mask]
    # inversa da CDF contínua: 1 - 0.3*exp(-a x), a=1e-5
    a = 1e-5
    u_tail = np.clip(u_tail, 0, 1 - eps)  # evita u=1 -> inf
    x[mask] = -np.log((1 - u_tail) / 0.3) / a
    return x

# amostragem da distribuicao
samples_model_4 = sample_mixture(10000, rng=42)


In [10]:
print(np.std(samples_model_4))

72535.42439580665


In [11]:
print(skewness(samples_model_4))

3.73796536700586


In [12]:
print(kurtosis(samples_model_4))

21.741356232728343


# 3.5

In [13]:
d = 1000

excess_loss_model_2 = [x for x in samples_model_2 if x > d]
excess_loss_model_4 = [x for x in samples_model_4 if x > d]

In [14]:
print(np.mean(excess_loss_model_2))

2540.4717627176374


In [15]:
print(np.mean(excess_loss_model_4))

101932.41997862978
